In [1]:
%matplotlib inline
import os

import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [2]:
os.getenv("DATASET_PATH")

'/Users/kayoonkim/Documents/winter-2023/data-literacy/city-recsys/streamlit/all-data-best-city.csv'

In [30]:
final_df = pd.read_csv(os.getenv("DATASET_PATH"), index_col=0)

In [31]:
final_df['city_ascii'] = final_df['city_ascii'].str.strip()

In [5]:
final_df.head()

,city_ascii,cost_live_rent_index,purchase_pow_index,country,mean_tmp,std_dev_temp,safety_index,pollution_index,trffic_min_index,lat,lng
0,Honolulu,80.4,74.4,United States,25.441667,1.837220,52.6,37.0,41.5,21.3294,-157.8460
1,Seattle,83.4,108.4,United States,11.316667,5.142662,45.7,34.7,46.9,47.6211,-122.3244
2,Canberra,69.1,69.1,Australia,13.141667,5.418396,72.0,26.1,43.3,-35.2931,149.1269
3,Adelaide,64.6,67.3,Australia,17.208333,4.416798,66.7,27.1,45.1,-34.9275,138.6000
4,Boston,82.0,106.6,United States,12.063636,8.341855,59.6,32.3,46.5,42.3188,-71.0852


In [6]:
final_df.columns

Index(['city_ascii', 'cost_live_rent_index', 'purchase_pow_index', 'country',
       'mean_tmp', 'std_dev_temp', 'safety_index', 'pollution_index',
       'trffic_min_index', 'lat', 'lng'],
      dtype='object')

In [7]:
#%pip install dash

In [61]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# initialize
app = dash.Dash(__name__)
dropdown_options = [{'label': col, 'value': col} for col in final_df.drop(columns=['city_ascii', 'country', 'lat', 'lng'])]

# app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='index-dropdown',
        options=dropdown_options,
        value='safety_index',  # initial index
        style={'width': '50%', 'margin': '10px'}
    ),
    dcc.Graph(
        id='world-map',
        style={'height': '90vh', 'width': '100vw'}  # full size
    )
])

# callback to update the visualization
@app.callback(
    Output('world-map', 'figure'),
    [Input('index-dropdown', 'value')]
)

def update_plot(selected_index):
    
    fig = px.scatter_geo(
        final_df, lat='lat', lon='lng', color=selected_index,
        color_continuous_scale='blues' if selected_index in ['safety_index', 'purchase_pow_index', 'trffic_min_index'] else 'reds',
        hover_name='city_ascii', labels={selected_index: selected_index.replace('_', ' ').title()} ,
        title=f'{selected_index.replace("_", " ").title()} Visualization', projection='natural earth',
    )

    return fig


if __name__ == '__main__':
    app.run_server(port=8000, host='127.0.0.1', debug=True, use_reloader=False)
